# Part I. ETL Pipeline for Pre-Processing the Files

## Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Pre-processing the files

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print("current working directory: %s" % os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

current working directory: /mnt/d/GitHub/data-modeling-with-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# check total rows
print("total rows: %s" % len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

total rows: 8056


In [4]:
# check the number of rows in your csv file
print("number of rows in csv file:", end=" ")
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

number of rows in csv file: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Initialize cluster and session

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create table

In [8]:
query = "CREATE TABLE IF NOT EXISTS app_history_1 "
query += "(artist text, song text, length double, session_id int, item_in_session int, \
PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### Insert data

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO app_history_1 (artist, song, length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into table

In [10]:
query = "SELECT * FROM app_history_1 WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("artist: {}, song: {}, length: {}".format(row.artist, row.song, row.length))

artist: Faithless, song: Music Matters (Mark Knight Dub), length: 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create table

In [11]:
query = "CREATE TABLE IF NOT EXISTS app_history_2 "
query += "(artist text, song text, session_id int, item_in_session int, user_id int, first_name text, last_name text, \
PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert data

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history_2 (artist, song, session_id, item_in_session, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[8]), int(line[3]), int(line[10]), line[1], line[4]))  

#### Validate the data model

In [13]:
query = "SELECT * FROM app_history_2 WHERE user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("artist: {}, song: {}, user: {} {}, item in section: {}".format(
            row.artist, row.song, row.first_name, row.last_name, row.item_in_session)
         )

artist: Down To The Bone, song: Keep On Keepin' On, user: Sylvie Cruz, item in section: 0
artist: Three Drives, song: Greece 2000, user: Sylvie Cruz, item in section: 1
artist: Sebastien Tellier, song: Kilometer, user: Sylvie Cruz, item in section: 2
artist: Lonnie Gordon, song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), user: Sylvie Cruz, item in section: 3


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create table

In [14]:
query = "CREATE TABLE IF NOT EXISTS app_history_3 "
query += "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert data

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history_3 (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Validate the data model

In [16]:
query = "SELECT * FROM app_history_3 WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("user: {} {}, user_id: {}".format(row.first_name, row.last_name, row.user_id))

user: Jacqueline Lynch, user_id: 29
user: Tegan Levine, user_id: 80
user: Sara Johnson, user_id: 95


## Drop the tables before closing out the sessions

In [17]:
query = "drop table app_history_1"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "drop table app_history_2"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "drop table app_history_3"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()